In [17]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
from splinter import Browser

In [18]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [19]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [20]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Retrieve all article slides
slides = soup.find_all('li',class_='slide')

In [21]:
# Find the title and text of each article and save to lists
news_titles = []
news_texts = []
for slide in slides:
    news_titles.append(slide.find('div',class_='content_title').text.strip())
    news_texts.append(slide.find('div',class_='article_teaser_body').text.strip())

## JPL Mars Space Images - Featured Image

In [22]:
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
html = browser.html
# Use splinter to click on the Full image button
browser.click_link_by_partial_text('FULL IMAGE')
html2 = browser.html
# Retrieve the link the image url
soup = BeautifulSoup(html2, 'lxml')
mars_images = soup.find('div',class_='fancybox-inner').find('img')

In [23]:
# Save the full image url
featured_image_url = "https://www.jpl.nasa.gov"+mars_images.get('src')

AttributeError: 'NoneType' object has no attribute 'get'

## Mars Weather

In [24]:
# Retrieve the latest Mars weather tweet
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)
html = browser.html
soup = BeautifulSoup(html, 'lxml')
mars_weather = soup.find('p',class_='tweet-text').text

## Mars Facts

In [25]:
import pandas as pd

In [26]:
# Use the read_html function in Pandas to scrape any tabular data from the page
url4 = 'https://space-facts.com/mars/'
tables = pd.read_html(url4)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [27]:
# Save and format the first dataframe in the tables list.
df = tables[0]
df = df.rename(columns={0:''})
df = df.rename(columns={1:'values'})
df.set_index('',inplace=True)
df

,values
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [28]:
# Use pandas to_html method to generate an HTML table from the DataFrame
html_table = df.to_html('table.html')
html_table

In [29]:
#!open table.html

## Mars Hemispheres

In [30]:
import time
hemispheres = ['Cerberus Hemisphere','Schiaparelli Hemisphere', 'Syrtis Major Hemisphere','Valles Marineris Hemisphere']

hemisphere_image_urls = []
for hemisphere in hemispheres:
    url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url5)
    button = browser.find_link_by_partial_text(hemisphere)
    button.click()
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    hemisphere_title=soup.find('section',class_='block metadata').find('h2').text
    image_div= soup.find('div',class_='downloads')
    hemisphere_image=image_div.ul.find_all('li')[1].a['href']
    hemisphere_image_urls.append({"title":hemisphere_title,"img_url":hemisphere_image})


In [31]:
hemisphere_image_urls


[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
  'title': 'Valles Marineris Hemisphere Enhanced'}]